# Exercise #4: neural networks primer (part 2)

Last time we covered

**Phase 1 : set it up**

 1. use case
 2. from logit to neural network
 3. nn structure: weights and biases
 3. activation function
 4. softmax
 5. application
 
To proceed we will need to run some code:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

#Load dataset
app = pd.read_csv("data/PrepApp.csv",index_col=False,sep='\t', encoding='utf-8')
app=app.set_index('track_name')

#Define functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def ReLU(x):
    return np.maximum(0, x)

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

#Prepae your data, starting with dummies for target var
encoder = LabelEncoder()
encoder.fit(app["user_rating"])
encoded_Y = encoder.transform(app["user_rating"])
dummy_y = np_utils.to_categorical(encoded_Y)
data = np.array(app)

X = data[:,:-1]
y = dummy_y.astype(int)

# We will feed in the 6th observation
inputs = np.array(X[5]).reshape( (35,1) )
targets = np.array(y[5]).reshape( (6,1) )

#Set up the NN structure
inputLayer = 35 
hiddenLayer = 50 
outputLayer = 6 

#Set weights and biases
np.random.seed(9)
limit = np.sqrt(6 / (inputLayer + outputLayer))
weightsInputToHidden = np.random.uniform(-limit, limit, (hiddenLayer, inputLayer))
weightsHiddenToOutput = np.random.uniform(-limit, limit, (outputLayer, hiddenLayer))
biasInputToHidden = np.ones( (hiddenLayer,1) ) 
biasHiddenToOutput = np.ones( (outputLayer,1) )

#Start the forward pass
hL_inputs = np.dot(weightsInputToHidden, inputs) + biasInputToHidden
hL_outputs = ReLU(hL_inputs)
oL_inputs = np.dot(weightsHiddenToOutput, hL_outputs) + biasHiddenToOutput
oL_outputs = softmax(oL_inputs)

#Check your results
print(oL_outputs)
print(y[5])

Using TensorFlow backend.


[[0.00635426]
 [0.01998603]
 [0.01393413]
 [0.00850893]
 [0.87939691]
 [0.07181974]]
[0 0 0 0 1 0]


# Phase 2: make it work # 
 - loss function
 - stochastic gradient descent
 - learning rate
 - momentum

## Loss function
### Logistic regression case
$$J(w) = \sum_{i=1}^{m} y^{(i)} \log P(y=1) + (1 - y^{(i)}) \log P(y=0)$$
Where P(y) represent the probability of a certain binary outcome

We will however consider another loss functin - cross entropy. It is used when output represents the probability of an outcome, i.e. when the output is a probability distribution. It is used as a loss function in neural networks which have softmax activations in the output layer.

### Cross-entropy 
Entropy (H(y)) is a term from Information Theory. It had a great impact on the field of communication and signifies the optimal number of bits to encode a certain information content ($y_i$ is the probability of the i-th event, symbol or in our case class):

$$H(y) = \sum_i y_i \log \frac{1}{y_i} = -\sum_i y_i \log y_i$$

Now the cross-entropy (H(y,y^)) is the number of bits we'll need if we encode symbols from  y using the wrong tool y^. Cross entropy is always bigger or equal to entropy. Mind that i stands for the number of classes. 

$$H(y, \hat{y}) = \sum_i y_i \log \frac{1}{\hat{y}_i} = -\sum_i y_i \log \hat{y}_i$$

Interestingly enough, the The KL divergence that you have encountered before in BADS (uplift random forest) is simply the difference between cross entropy and entropy:
$$\mbox{KL}(y~||~\hat{y})
= \sum_i y_i \log \frac{1}{\hat{y}_i} - \sum_i y_i \log \frac{1}{y_i}
= \sum_i y_i \log \frac{y_i}{\hat{y}_i}$$

We would be calculating the cross-entropy for every vector of true/estimated probabilities and averaging it over the sample or batch (more about it later) - this will be our loss function *L* that we will ultimately want to minimise (class i, smaple j):

$$L=-\frac{1}{N}\sum_j \sum_i y_i \log(yhat)$$

In [2]:
#CE example
#A concrete example is the best way to explain the purely mathematical form of CE. Suppose you have a weirdly shaped four-sided dice (yes, I know the singular is really "die").
#Using some sort of intuition or physics, you predict that the probabilities of the four sides are (0.20, 0.40, 0.30, 0.10). 
# Then you roll the dice many thousands of times and determine that the true probabilities are (0.15, 0.35, 0.25, 0.25). 
# The CE error for your prediction is:
-1.0 * [ ln(0.20) * 0.15 + ln(0.40) * 0.35 + ln(0.30) * 0.25 + ln(0.10) * 0.25 ] =
-1.0 * [ (-1.61)(0.15) + (-0.92)(0.35) + (-1.20)(0.25) + (-2.30)(0.25) ] =
1.44
#https://visualstudiomagazine.com/articles/2017/07/01/cross-entropy.aspx

SyntaxError: invalid syntax (<ipython-input-2-9b1662d53cce>, line 6)

<img src="softmax.png" alt="ffff" style="width: 600px">

Source: https://www.ritchieng.com/machine-learning/deep-learning/neural-nets/

In [ ]:
true = y[5]
yhat=np.round(oL_outputs, 3) #that's our WX+b run through softmax
print(yhat,yhat.shape)


In [ ]:
L=-sum(true*np.log(yhat))
L

In [ ]:
#print(true, true.shape)
#print(yhat,yhat.shape)
#Let's define loss function

L=-sum(true*np.log(yhat))
print(L) #doesnt look quite right...let's see what went wrong

In [ ]:
true

In [ ]:
true*np.log(yhat)

In [ ]:
true*np.log(yhat.flatten())

In [ ]:
L=-sum(true*np.log(yhat.flatten()))
print(L)
 # we are summing over classes, not over obs, as we have only one observation!
#the loss function values seems right now, let's see how we can improve it    

This is the number we want the coomputer to minimize! But how?
We can not change our input, we cannoo change our labels, what we can **update weights and biases** - these are the parameters of our neural network. So we need to change our **W, M, b and v** see if loss function decreases.
NB: number of neurons in the hidden layer AND number of hidden layers are hyperparameters and certainly can be experimented with, we will discuss it further down the course.

The most efficient way to move around the function slope is to find the derivative.

In our naive example N=1, as we are only dealing with 1 batch (j=1), while we have 6 classes (i=6).

But i an j would potentially go into thousands, calculating the derivative of the loss function will become extremely hard. 

And that was the point when everybody almost gave up on neural networks. Spoiler alert: the solutions were **stochastic gradient descent and backpropagation**.


In [1]:
%%html
<iframe src="https://giphy.com/embed/8tvzvXhB3wcmI" width="1000" height="400" frameBorder="0" class="giphy-embed" allowFullScreen></iframe>
<p><a href="https://giphy.com/gifs/deep-learning-8tvzvXhB3wcmI">via GIPHY</a></p>

In [ ]:
#We will try to find the local minimum of (x-4)^2 function. In this case gradient is a simple derivative

cur_x = 10 
rate = 0.1 # Learning rate
precision = 0.000001 #This tells us when to stop the algorithm
previous_step_size = 1 #
max_iters = 40 # maximum number of iterations
iters = 0 #iteration counter
df = lambda x: 2*(x-4) #Gradient of our function '

while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x #Store current x value in prev_x
    cur_x = cur_x - rate * df(prev_x) #Grad descent
    previous_step_size = abs(cur_x - prev_x) #Change in x
    iters = iters+1 #iteration count
    print("Iteration",iters,"\nX value is",cur_x) #Print iterations
    
print("The local minimum occurs at", cur_x)

#https://towardsdatascience.com/implement-gradient-descent-in-python-9b93ed7108d1

## Gradient
The gradient ( $\nabla$) is a vector operation which operates on a scalar function to produce a vector whose magnitude is the maximum rate of change of the function at the point of the gradient and which is pointed in the direction of that maximum rate of change. 

Well, put in easier terms, gradient is a **vector of partial derivatives**. Why would we need it? Because we need the derivative of this:
$$L=-\frac{1}{N}\sum\sum y\log(softmax(M(relu(Wa+b))+v))$$


We want to adjust every estimation of y (yhat) so as to minimize the loss function. For that we would deduct a gradient of the loss function from it and continue doing these iterations until we reach the local minimum. 
$$L_{t+1} =L_t - \eta\nabla L_t(W,M,b,v)$$

You can think of of gradient as a list of directionsfor improvement (of course, imagining moving in 10000 directions is hard):
$$ \nabla L(W,M,b,v)=\begin{bmatrix}  \frac{dL}{dW}\\\frac{dL}{dM}\\ \frac{dL}{db}\\ \frac{dL}{dv}\end{bmatrix}$$
Normally it would be a result of averaging over all observations that went through with forward pass, but again- we have just one this time.

## Derivation ##

refer to the video

## Updating weights

In [138]:
def loss_derivative(output, y): #check the derivation if interested https://sefiks.com/2017/12/17/a-gentle-introduction-to-cross-entropy-loss-function/
    return output - y

def ReLU_derivative(x):
    return (x>0)*1.0

In [161]:
output_error=loss_derivative(oL_outputs, targets)
#output_error = (oL_outputs - targets) / oL_outputs.shape[0]
output_error

array([[ 0.00635426],
       [ 0.01998603],
       [ 0.01393413],
       [ 0.00850893],
       [-0.12060309],
       [ 0.07181974]])

In [162]:
#hidden_error=ReLU_derivative(np.dot(weightsHiddenToOutput.T, output_error))
hidden_error=np.dot(weightsHiddenToOutput.T, output_error)

hidden_error[hL_outputs <= 0] = 0 # instead of relu derivative

In [163]:
gradient_HiddenToOutput = np.dot(output_error, np.transpose(hL_outputs))
gradient_HiddenToOutput.shape

(6, 50)

In [164]:
gradient_InputToHidden = np.dot(hidden_error,np.transpose(inputs))
gradient_InputToHidden.shape

(50, 35)

In [165]:
learning_rate= 0.01

weightsHiddenToOutput -= learningRate *gradient_HiddenToOutput
weightsInputToHidden -= learningRate * gradient_InputToHidden

## Stochastic gradient descent and backpropagation##

Apart from derivatives, avereging over the whole training dataset might be extremely costly. That is why the **stochastic** gradient descent was introduced. It basically means that instead of using the full training set, the algorithm will only use a certain  random **batch** (size of this batch is a hyperparameter like the number of neurons). This introduces a certain "slopinness" to the process but allows to run the **backpropagation** much faster and after many iterations we expect to converge to the sample parameters anyway. Anpther important nption is an **epoch** - it means the amount of time the full dataset will be ran through the NN.

Backpropagation is an algorithm for computing the gradient in multidimensional space.

<img src="nn_color.png" alt="Backpropagation" style="width: 800px;"/>

In [167]:
learning_rate= 0.01
epochs = 10 # here is how many times we want to go through data
iteration = 0


while iteration < epochs:
    
    # Put all the steps done before in a loop
    # Mind that we are not reinitiating the weights!
    for i in range(X.shape[0]):  # we are not using the batches here, we put in the whole dataset
   
        inputs = np.array(X[i]).reshape( (35,1) )
        targets= y[i]
        targets = np.array(y[i]).reshape( (6,1) )
        

        hL_inputs = np.dot(weightsInputToHidden, inputs) + biasInputToHidden
        hL_outputs = ReLU(hL_inputs)
        oL_inputs = np.dot(weightsHiddenToOutput, hL_outputs) + biasHiddenToOutput 
        oL_outputs = softmax(oL_inputs)

        #L=-sum(targets*np.log(np.round(oL_outputs, 3)))
        #print(L)
        output_error=loss_derivative(oL_outputs, targets)
        hidden_error=np.dot(weightsHiddenToOutput.T, output_error)
        hidden_error[hL_outputs <= 0] = 0

        gradient_HiddenToOutput = np.dot(output_error, np.transpose(hL_outputs))
        gradient_InputToHidden = np.dot(hidden_error,np.transpose(inputs))
       
        weightsHiddenToOutput -= learningRate *gradient_HiddenToOutput
        weightsInputToHidden -= learningRate *gradient_InputToHidden
       
        

    iteration += 1

In [169]:
# scorecard of the network
confusion_matrix = np.zeros((6,6), dtype="int" )

# go through all the observations in the data set
for i in range(len(y)):
    inputs = np.array(X[i]).reshape( (35,1) )

    hL_inputs = np.dot(weightsInputToHidden, inputs) + biasInputToHidden
    hL_outputs = ReLU(hL_inputs)
    oL_inputs = np.dot(weightsHiddenToOutput, hL_outputs) + biasHiddenToOutput
    oL_outputs = softmax(oL_inputs)

    label = np.argmax(oL_outputs)
    confusion_matrix[label, y[i]] += 1

In [170]:
confusion_matrix

array([[1330, 1330,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0],
       [   9,    9,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0],
       [5855, 5855,    0,    0,    0,    0],
       [   3,    3,    0,    0,    0,    0]])

## learning rate

Learning rate is a **hyper-parameter** that controls how much we are adjusting the weights of our network with respect the loss gradient. 
Lower LR takes more time but allows better allocation of local minimum, higher LR allows faster calculations but drastic jumps do not always yield good results. 
        $$ newWeights=OldWeights - learningRate *gradientOfOldWeights$$ 

<img src="http://cs231n.github.io/assets/nn3/learningrates.jpeg" alt="Drcng" style="width: 400px;"/>Img Credit: cs231n

One can improve the results of computations significantly if learning rate is set well. However, learning rate might not remain the same throughout the training. The concpet of cyclical learning rate was introduced by Leslie N.Smith in 2015, it conveys a certain schedule when the LR starts with small values and increases (either linearly or exponentially) at each iteration. Learning rate decay would provide an alternative, it would bare the same problem though - when to decay the LR (step decay, exponential decay, others). In practice, step decay is preferred by many practitioners as hyperparameters it involves (the fraction of decay and the step timings in units of epochs) are more interpretable. 

learning_rate = 0.1
decay_rate = learning_rate / epochs

We will learn more about hte applications during the next tutorial. Meanwhile, there is another hyperparameter that is often overseen but may yield very good convergence results.

## momentum

## 2.7 Momentum

$$\Delta  W_{i} = -learningRate  \frac{\partial L}{\partial W} + \mu  \Delta W_{i-1}$$


Second part that contains $\mu$ is a momentum term here (or coefficient), that defines the effect of the accumulated past gradient (we are aking an exponentially weighted moving average of accumulated gradient). You can think of it as a certain velocity control mechanism. When we reach flatter areas, it will increase the speed of convergence, while dampening oscillations when reaching high curvatures. If learning rate measures how much the current situation affects the next step, while momentum measures how much past steps affect the next step. 

Conventional values to set for momentum is 0.5 increasing to 0.9, in case of cross validation can be set to values such as [0.5, 0.9, 0.95, 0.99]

**Nesterov Momentum** is a slightly different version of the momentum update that has recently been gaining popularity. It is set as a metaparameter in basic Keras application that we will see in the next tutorial. In simplified terms, Nesterov momentum gives gradient a better 'nudge' as it containes a 'lookahead' information. 
